In [1]:
from imports import *
import staircase as sc
%matplotlib inline
import operator
import warnings
from hotstepper.Utils import Utils
import hotstepper.fastbase as fb
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"../data/hotel_stays.csv", parse_dates=['check_in', 'check_out'], dayfirst=True)

#df = df_cc.loc[df_cc.call_type == 'Type 1']

df.shape

(13602, 6)

In [3]:
#%%timeit -r 3 -n 3
hotel_stays = Steps.read_dataframe(df,'check_in', 'check_out')
#aborted_type1 = Steps.read_dataframe(df,'arrived','aborted')
#duration_type1 = Steps.read_dataframe(df,'answered','complete')

In [4]:
queue = sc.Stairs(use_dates=True).layer(df.check_in, df.check_out)

In [ ]:
st = np.array([[s._start_ts,s._direction,s._weight] for s in hotel_stays],dtype=float)
p = hotel_stays.step_keys()
fig,ax = plt.subplots(figsize=(14,6))
t = np.asarray(list(map(Utils.get_ts, p)))
x = np.asarray(fb.fast_steps_heaviside().step(st,t,1))
#x2 = np.asarray(fb.fast_steps_logit().step(st,t,500000))
ax.step(p,x)
#ax.step(p,x2,color='r')

In [ ]:
timespan = {'lbound':pd.Timestamp(2020,2,15,10),'ubound':pd.Timestamp(2020,2,16)}
ax = hotel_stays.plot(color='g',size=(36,6))
hotel125 = (hotel_stays>150)
#hotel125.plot(ax=ax,color='r')

#ax2 = (hotel_stays>80).plot(color='g')
hotel125.normalise().plot(ax=ax.twinx(),color='r')

#aborted_type1.plot(ax=ax,color='r')
#duration_type1.plot(ax=ax,color='blue')

In [5]:
#p = hotel_stays.step_keys()
p=np.arange(pd.Timestamp(2016,1,1),pd.Timestamp(2017,1,1),pd.Timedelta(minutes=20)).astype(pd.Timestamp)

In [7]:
%%timeit -r 3 -n 3
st = np.array([[s._start_ts,s._direction,s._weight] for s in hotel_stays],dtype=float)
t = np.asarray(list(map(Utils.get_ts, p)))
x = np.asarray(fb.fast_steps_heaviside().step(st,t,1))

1.43 s ± 16.5 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [8]:
%%timeit -r 3 -n 3
st = np.array([[s._start_ts,s._direction,s._weight] for s in hotel_stays],dtype=float)
t = np.asarray(list(map(Utils.get_ts, p)))
x2 = np.asarray(fb.fast_steps_logit().step(st,t,500))

2.06 s ± 45.7 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [6]:
%%timeit -r 3 -n 3
queue(p)

169 ms ± 4.45 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
